# NeuroConv - Simple Tutorial

In [ ]:
from pathlib import Path
from datetime import datetime

from pynwb import NWBHDF5IO

from neuroconv import NWBConverter, RecordingTutorialInterface, SortingTutorialInterface

## Define conversion class and its internal data interfaces (*i.e.*, the name of format)
### For a full list of supported formats, [see this list](https://nwb-conversion-tools.readthedocs.io/en/conversion_guide/converting_data_to_nwb.html), or [make your own data interface](https://nwb-conversion-tools.readthedocs.io/en/conversion_guide/data_interface.html)

In [ ]:
class TutorialNWBConverter(NWBConverter):
    data_interface_classes = dict(
        RecordingTutorial=RecordingTutorialInterface,
        SortingTutorial=SortingTutorialInterface
    )

## Construct arguments for the converter class and run conversion

In [ ]:
# Custom parameters for simulated toy data
duration = 10.  # Seconds
num_channels = 4
num_units = 10
sampling_frequency = 30000.  # Hz

stub_test = False  # Truncates data write for faster quality checking

output_file = "NWBConversionToolsSimpleTutorial.nwb"

# Input arguments to each interface
# For actual data formats, these arguments are typically file or folder paths to the data
source_data = dict(
    RecordingTutorial=dict(
        duration=duration,
        num_channels=num_channels,
        sampling_frequency=sampling_frequency
    ),
    SortingTutorial=dict(
        duration=duration,
        num_units=num_units,
        sampling_frequency=sampling_frequency
    )
)

# Initialize converter
converter = TutorialNWBConverter(source_data=source_data)

# Get metadata from source data
# For actual data formats, this generally pulls informatin from the header files for each interface
metadata = converter.get_metadata()

# User-input metadata
metadata["NWBFile"]["session_description"] = "NWB Conversion Tools tutorial."
metadata["NWBFile"]["experimenter"] = ["My name"]
metadata["NWBFile"]['session_start_time'] = datetime.now().astimezone().strftime("%Y-%m-%dT%H:%M:%S")
metadata["Subject"] = dict(subject_id="Name of imaginary testing subject (required for DANDI upload)")

# Conversion options for each interface
# For actual data formats, these can vary widely - read the docstring for the interface you want to use by entering
#     import neuroconv
#     neuroconv.NameOfDataInterface.run_conversion?
conversion_options = dict(
    RecordingTutorial=dict(stub_test=stub_test),
    SortingTutorial=dict()
)

# Run conversion
converter.run_conversion(
    metadata=metadata, 
    nwbfile_path=output_file, 
    save_to_file=True,  # If False, this instead returns the NWBFile object in memory
    overwrite=True,  # If False, this appends an existing file
    conversion_options=conversion_options
)

## View NWBFile with widgets

In [ ]:
from nwbwidgets import nwb2widget

io = NWBHDF5IO(output_file, "r", load_namespaces=True)

nwbfile = io.read()

nwb2widget(nwbfile)